Lista 02 de Mineração de Dados

In [ ]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving fakeTelegram.BR_2022.csv to fakeTelegram.BR_2022.csv


# item a) Ler o dataset fakeTelegram.BR_2022.csv

In [ ]:
import pandas as pd

df = pd.read_csv('fakeTelegram.BR_2022.csv')
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637



# item b) Remova os trava-zaps.



In [ ]:
df = df[~df['trava_zap']].reset_index(drop=True)
print(f"Total de linhas que sobraram: {len(df)}")

Total de linhas que sobraram: 557570


In [ ]:
! pip install pyarrow
! pip install duckdb

# item c) Exportar os dados para um arquivo Parquet

In [ ]:
import pyarrow
df.to_parquet('fakeTelegram.BR_2022.parquet')
df = pd.read_parquet('fakeTelegram.BR_2022.parquet')

# item d) Exportar os dados para o DuckDB.

In [ ]:
import duckdb

conexao = duckdb.connect()
conexao.register('fake_telegram', df)

#item e) Utlizando o DuckDB recupere:

In [ ]:
# 1- A quantidade de mensagens
resultado = conexao.execute("SELECT COUNT(id_message) FROM fake_telegram").fetchone()
print("Número de linhas:", resultado[0])

Número de linhas: 557570


In [ ]:
#2- A quantidade de usuários
resultado = conexao.execute("SELECT COUNT(id_member_anonymous) FROM fake_telegram").fetchone()
print("Número de usuários:", resultado[0])

Número de usuários: 234233


In [ ]:
#3- A quantidade de grupos
resultado = conexao.execute("SELECT COUNT(id_group_anonymous) FROM fake_telegram").fetchone()
print("Número de grupos:", resultado[0])

Número de grupos: 557570


In [ ]:
#4- Quantidade de mensagens que possuem apenas texto
resultado = conexao.execute("""SELECT COUNT (message_type)
                            FROM fake_telegram
                            WHERE message_type LIKE '%Texto%'""").fetchone()
print("Número de mensagens que possuem apenas texto:", resultado[0])

Número de mensagens que possuem apenas texto: 224966


In [ ]:
#5- Quantidade de mensagens contendo mídias
resultado = conexao.execute("""SELECT COUNT (has_media)
                                FROM fake_telegram
                                WHERE has_media = True""").fetchone()
print("Número de mensagens contendo mídias:", resultado[0])

Número de mensagens contendo mídias: 332604


In [ ]:
#6- Quantidade de mensagens por tipo de mídia (jpg, mp4 etc)
resultado = conexao.execute("""SELECT media_type, COUNT (media_type) AS mensagens
                              FROM fake_telegram
                              GROUP BY media_type
                              ORDER BY mensagens DESC""").fetchdf()
resultado

,media_type,mensagens
0,image/jpg,200440
1,url,100856
2,video/mp4,18497
3,application/vnd.android.package-archive,7159
4,application/pdf,2850
...,...,...
63,image/45dbf1564a99d025ade111ea1dd2add5,1
64,image/xps,1
65,application/x-partial-download,1
66,application/binary,1


In [ ]:
#12- As 30 URLs que mais se repetem (mais compartilhadas)
resultado = conexao.execute(""" SELECT media_url, COUNT (media_url) AS total
                                FROM fake_telegram
                                GROUP BY media_url
                                ORDER BY total
                                DESC LIMIT 30""").fetchdf()
resultado

,media_url,total
0,t.me/alexeconomia,4159
1,https://youtube.com/c/especulandoosfatosoficial,1607
2,t.me/fimtaproximo,1581
3,t.me/+EWlGMatRZGg3OTlh,1419
4,https://youtu.be/qbTzhB0akt8,1160
5,https://youtu.be/zDuOoyhyN-4,1022
6,t.me/+ewZIPdZ42vEyNzJh,772
7,https://t.me/canalselvabrasiloficial,709
8,https://youtu.be/4DHk9KZ01HM,660
9,T.me/Arthur_Senna_Trader2,640


In [ ]:
#13- Os 30 domínios que mais se repetem (mais compartilhados);
resultado = conexao.execute("""
    SELECT
        REGEXP_EXTRACT(media_url, '^(?:https?://)?(?:www\\.)?([^/]+)') AS dominio,
        COUNT(*) AS total
    FROM fake_telegram
    WHERE media_url IS NOT NULL AND TRIM(media_url) != ''
    GROUP BY dominio
    ORDER BY total DESC LIMIT 30
""").fetchdf()
resultado

,dominio,total
0,https://youtu.be,52709
1,https://t.me,10615
2,t.me,9608
3,https://www.youtube.com,8313
4,https://terrabrasilnoticias.com,8282
5,https://youtube.com,5960
6,https://www.instagram.com,5783
7,https://www.jornaldacidadeonline.com.br,4829
8,https://twitter.com,4389
9,https://gazetabrasil.com.br,1779


In [ ]:
#14- Os 30 usuários mais ativos
resultado = conexao.execute("""SELECT id_member_anonymous, COUNT (id_member_anonymous)
                            AS total FROM fake_telegram
                            GROUP BY id_member_anonymous
                            ORDER BY total
                            DESC LIMIT 30""").fetchdf()
resultado

,id_member_anonymous,total
0,abe534d581ec6d552243d6955d3c3cd8,12289
1,1665e22b0f564cd46d343f7677014821,5452
2,1ac091b8ed5c4e42383f1b4ff4cc9b2d,5060
3,c743967449a387ad2c1c7e03b2c45b36,3019
4,e7998863ac2a40086657fab4a6b463c9,1928
5,e8fd8fee8c39342d37993775da7756d5,1706
6,2f4be6244ede15b46e8329a2c975be30,1620
7,d49f81df0c75d1d72bee6c5b2d707da0,1571
8,4f7d493f0f6222d56e5b19a4f7c336cc,1447
9,773b9bd5b02a2e96f9d732c29bfcb663,1411


In [ ]:
#15- Os 30 usuários que mais compartilharam texto
resultado = conexao.execute("""SELECT id_member_anonymous, COUNT (id_member_anonymous)
                            AS total FROM fake_telegram
                            WHERE message_type = 'Texto'
                            GROUP BY id_member_anonymous
                            ORDER BY total DESC
                            LIMIT 30""").fetchdf()
resultado

,id_member_anonymous,total
0,abe534d581ec6d552243d6955d3c3cd8,12289
1,c743967449a387ad2c1c7e03b2c45b36,2867
2,1665e22b0f564cd46d343f7677014821,2480
3,d49f81df0c75d1d72bee6c5b2d707da0,1571
4,f233cf8b1d4ede06f32199a7e0081bf5,1022
5,e8fd8fee8c39342d37993775da7756d5,699
6,a378e9743fa3ca297df321cfa0e7cf6a,598
7,7696d5103cdb8ac352d748a1db1126b0,560
8,26b496125c8b6773bce453e14f172430,502
9,7d41f1a2df245b0cc870f546c2aead0b,483


In [ ]:
#16- Os 30 usuários que mais compartilharam mídias
resultado = conexao.execute("""SELECT id_member_anonymous, COUNT (id_member_anonymous)
                            AS total FROM fake_telegram
                            WHERE has_media = 'True'
                            GROUP BY id_member_anonymous
                            ORDER BY total DESC
                            LIMIT 30""").fetchdf()
resultado

,id_member_anonymous,total
0,1ac091b8ed5c4e42383f1b4ff4cc9b2d,4632
1,1665e22b0f564cd46d343f7677014821,2972
2,e7998863ac2a40086657fab4a6b463c9,1498
3,2f4be6244ede15b46e8329a2c975be30,1489
4,4f7d493f0f6222d56e5b19a4f7c336cc,1359
5,c052c859b42c5a1923c22f5a201de746,1289
6,773b9bd5b02a2e96f9d732c29bfcb663,1167
7,56b8359fd127312651b80b8ed8030085,1079
8,e8fd8fee8c39342d37993775da7756d5,1007
9,3e49fd40fd973ee1b8f1a6d58feb4a54,965


In [ ]:
#17- As 30 mensagens mais compartilhadas
resultado = conexao.execute("""SELECT text_content_anonymous, COUNT (text_content_anonymous)
                            AS total FROM fake_telegram
                            GROUP BY text_content_anonymous
                            ORDER BY total
                            DESC LIMIT 30""").fetchdf()
resultado

,text_content_anonymous,total
0,This community was blocked in Brazil following...,17422
1,Rough_sex🙈,1134
2,Anal sex🙈,1118
3,سکس مردان ازبک با زن انگلیسی با این vpn از سای...,1019
4,https://youtu.be/qbTzhB0akt8,758
5,https://youtu.be/zDuOoyhyN-4,712
6,فیلم سوپر با زیرنویس فارسی ببین😍😍\nبا این فیلت...,632
7,Rough😱,570
8,Foto de Nélia Barros,548
9,We had no choice but to remain in the shadows....,480


In [ ]:
#18- As 30 mensagens mais compartilhadas em grupos diferentes
resultado = conexao.execute("""SELECT text_content_anonymous, COUNT (DISTINCT id_group_anonymous)
                            AS total FROM fake_telegram
                            WHERE text_content_anonymous IS NOT NULL
                            GROUP BY text_content_anonymous
                            ORDER BY total DESC
                            LIMIT 30""").fetchdf()
resultado

,text_content_anonymous,total
0,This community was blocked in Brazil following...,59
1,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,36
2,Hoje às 18 horas no canal Inteligência Ltda do...,35
3,[USER],32
4,https://youtu.be/zDuOoyhyN-4,31
5,https://youtu.be/qbTzhB0akt8,30
6,Ajudem a subir a #JanonesAmigoDePedofilo\n\nPo...,29
7,OS PTISTAS NÃO QUER QUE AGENTE MOSTRE QUE VOTO...,29
8,*ATENÇÃO* \n🚨🚨🚨🚨🚨🚨🚨🚨🚨\n\n*CHAMADA URGENTE* Pat...,29
9,"DE NADA IRÁ ADIANTAR FISCALIZAR AS URNAS, SE N...",29


In [ ]:
#19- Mensagens idênticas compartilhadas pelo mesmo usuário (e suas quantidades)
resultado = conexao.execute("""SELECT id_member_anonymous, text_content_anonymous, COUNT (text_content_anonymous)
                            AS total FROM fake_telegram
                            WHERE text_content_anonymous IS NOT NULL AND id_member_anonymous IS NOT NULL
                            GROUP BY id_member_anonymous, text_content_anonymous
                            ORDER BY total DESC """).fetchdf()
resultado


,id_member_anonymous,text_content_anonymous,total
0,f233cf8b1d4ede06f32199a7e0081bf5,This community was blocked in Brazil following...,838
1,c743967449a387ad2c1c7e03b2c45b36,This community was blocked in Brazil following...,532
2,7696d5103cdb8ac352d748a1db1126b0,This community was blocked in Brazil following...,530
3,1665e22b0f564cd46d343f7677014821,Bem vindo(a) ao grupo Ipirá Notícias. \n\nComp...,357
4,a398999c55f8d6f0c65760522ae12e45,This community was blocked in Brazil following...,308
...,...,...,...
138621,f1d6850eb31bdce293399ebb0e6e1739,"Brasil Sem Medo - TSE ""lava as mãos"" e diz que...",1
138622,beb77036ff26e78865da19310ac95169,É XANDÃO..XANDÃO ACHO QUE AGORA A CASA CAIU PA...,1
138623,8f7bd84a25f6cc58c013828e38ba339d,https://youtu.be/RIL5ZWNugZk,1
138624,8f7bd84a25f6cc58c013828e38ba339d,https://youtu.be/0RA0nZJc_QU,1


In [ ]:
#20- Mensagens idênticas compartilhadas pelo mesmo usuário em grupos distintos (e suas quantidades)
resultado = conexao.execute("""SELECT id_member_anonymous, text_content_anonymous, COUNT (DISTINCT id_group_anonymous)
                            AS total FROM fake_telegram
                            WHERE text_content_anonymous IS NOT NULL AND id_member_anonymous IS NOT NULL
                            GROUP BY id_member_anonymous, text_content_anonymous
                            ORDER BY total DESC """).fetchdf()
resultado

,id_member_anonymous,text_content_anonymous,total
0,a7840e7844020149e197272748965862,ALERTA\nESSE GRUPO FOI CRIADO PELA ESQUERDA CO...,16
1,326d0a2f1bc5a1ede446fcf5dc31ff2e,💣💥((((( URGENTÍSSIMO )))))💥💣\nESSE GRUPO É UMA...,14
2,326d0a2f1bc5a1ede446fcf5dc31ff2e,💣💣💣💥💥🔰BOMBA BOMBA🔰💥💥💣💣💣\nVCS ESTÃO ACHANDO QUE...,12
3,29dda76df3384c28a07e8bad1ee9ceed,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,11
4,dc60c84ed8dec03b667c577e124c5540,INTERCESSÃO PELO BRASIL!!\n\nChegou a hora da ...,10
...,...,...,...
138621,92a98298316caf5bdbd65acfc20908a7,Bolsonaro faz duro discurso sobre possível fra...,1
138622,92a98298316caf5bdbd65acfc20908a7,Esse sujeito precisa ser protegido por ser tes...,1
138623,4e0e712955de6618a7def562bdcb87b1,por favor confirme mi transacción,1
138624,95614f2b3563171b240592587102604f,Precisamos de mais 1027804 assinaturas. Você p...,1


In [ ]:
#22- As 30 mensagens mais positivas (distintas)
resultado = conexao.execute("""SELECT DISTINCT text_content_anonymous, score_sentiment
                            FROM fake_telegram
                            ORDER BY score_sentiment DESC
                            LIMIT 30""").fetchdf()
resultado

,text_content_anonymous,score_sentiment
0,"Shemoneh Esreh – Amidá – A Grande Oração\n\n ,...",0.9992
1,"Pensa direita, não é correto compartilhar um v...",0.9990
2,"MENTIROSA!!!\n\n\nPensa direita, não é correto...",0.9990
3,"בונה ירושלים, Bo'ne Yetrushalayim - Benção da ...",0.9989
4,Os Sete Samurais \n \nQuando falamos sobre For...,0.9988
5,MANIFESTO À NAÇÃO BRASILEIRA\n\nEM DEFESA DO B...,0.9987
6,BOM DIA A TODOS. ENTENDO QUE ESTAMOS VIVENDO D...,0.9986
7,"Olá ,\n\nUM AMIGO ME PERGUNTOU EM QUEM EU VOTE...",0.9986
8,\nA Viva Esperança\n\nDepois das saudações in...,0.9985
9,Uma forma bem legal\nPara quem ainda não sabe ...,0.9985


In [ ]:
#23- As 30 mensagens mais negativas (distintas);
resultado = conexao.execute("""SELECT DISTINCT text_content_anonymous, score_sentiment
                            FROM fake_telegram
                            ORDER BY score_sentiment
                            LIMIT 30""").fetchdf()
resultado

,text_content_anonymous,score_sentiment
0,CAPITÃOZINHO DE MERDA 💩\nCAPITÃOZINHO DE MERDA...,-1.0000
1,fraude fraude fraude fraude fraude fraude frau...,-0.9999
2,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
3,ME CÚ NO PAU DO BOLSONARO \nME CÚ NO PAU DO BO...,-0.9999
4,"Gente, a Zambelli nos abandonou! Traiu a nossa...",-0.9999
5,ai ta na rua da um joinha 👍Quem ai ta na rua d...,-0.9999
6,"E nós aqui nos sacrificando, aguentando sol, c...",-0.9999
7,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
8,Sobre a morte como política global e a necessi...,-0.9998
9,a na rua da um joinha 👍Quem ai ta na rua da um...,-0.9998


In [ ]:
#24- O usuário mais otimista
resultado = conexao.execute("""SELECT id_member_anonymous, SUM(score_sentiment)
                            AS total
                            FROM fake_telegram
                            WHERE id_member_anonymous IS NOT NULL
                            GROUP BY id_member_anonymous
                            ORDER BY total DESC
                            LIMIT 1""").fetchdf()
resultado

,id_member_anonymous,total
0,1665e22b0f564cd46d343f7677014821,3105.8447


In [ ]:
#25- O usuário mais pessimista
resultado = conexao.execute("""SELECT id_member_anonymous, SUM(score_sentiment)
                            AS total
                            FROM fake_telegram
                            WHERE id_member_anonymous IS NOT NULL
                            GROUP BY id_member_anonymous
                            ORDER BY total
                            LIMIT 1""").fetchdf()
resultado

,id_member_anonymous,total
0,c743967449a387ad2c1c7e03b2c45b36,-333.2689


In [ ]:
#26- As 30 maiores mensagens
resultado = conexao.execute("""SELECT text_content_anonymous, LENGTH(text_content_anonymous)
                            AS total
                            FROM fake_telegram
                            WHERE text_content_anonymous IS NOT NULL
                            ORDER BY total DESC
                            LIMIT 30""").fetchdf()
resultado

,text_content_anonymous,total
0,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOO...,4096
1,Trecho do livro “Prática da Conspiração A Agen...,4096
2,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096
3,COMO O SOCIALISMO DESTRUIU A VENEZUELA EM POUC...,4096
4,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096
5,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOO...,4096
6,"1: Aconteceu também naqueles dias, quando não ...",4096
7,Trecho do livro “Prática da Conspiração A Agen...,4096
8,Trecho do livro “Prática da Conspiração A Agen...,4096
9,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096


In [ ]:
#27- As 30 menores mensagens
resultado = conexao.execute("""SELECT text_content_anonymous, LENGTH(text_content_anonymous)
                            AS total
                            FROM fake_telegram
                            WHERE text_content_anonymous IS NOT NULL
                            ORDER BY total
                            LIMIT 30""").fetchdf()
resultado

,text_content_anonymous,total
0,/,1
1,🤔,1
2,Q,1
3,🌩,1
4,🤭,1
5,🤭,1
6,🎯,1
7,😂,1
8,🌹,1
9,🤔,1


In [ ]:
#28- O dia em que foi publicado a maior quantidade de mensagens
resultado = conexao.execute("""SELECT CAST (date_message AS DATE) AS Dia, COUNT(id_message)
                            AS total
                            FROM fake_telegram
                            GROUP BY Dia
                            ORDER BY total DESC
                            LIMIT 1""").fetchdf()
resultado

,Dia,total
0,2022-10-02,25243


In [ ]:
#29- As mensagens que possuem as palavras “FACÇÃO” e “CRIMINOSA”
resultado = conexao.execute("""SELECT (text_content_anonymous)
                            FROM fake_telegram
                            WHERE text_content_anonymous ILIKE '%FACÇÃO%'
                            AND text_content_anonymous ILIKE '%CRIMINOSA%' """).fetchdf()
resultado

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,text_content_anonymous
0,"Pensem!!!!!!'Se Houve fraude ,Nada foi feito ..."
1,#lulaladrãoseulugarénaprisão \n#PTFACÇÃOCRIMIN...
2,"Se depois de ler esse resumo do Plano do PT, v..."
3,"Se depois de ler esse resumo do Plano do PT, v..."
4,É exatamente isso que a facção criminosa CHAM...
...,...
134,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
135,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
136,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
137,Agressões covardes é termos uma facção crimino...


In [ ]:
#30- . As mensagens que possuem a palavra “SEGURANÇA”
resultado = conexao.execute("""SELECT (text_content_anonymous)
                            FROM fake_telegram
                            WHERE text_content_anonymous ILIKE '%SEGURANÇA%'  """).fetchdf()
resultado

,text_content_anonymous
0,🌳 Mato Grosso intensifica ações contra crimes ...
1,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
2,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
3,No Telegram entrem em Configurações > Privacid...
4,No Telegram entrem em Configurações > Privacid...
...,...
2795,Profissão médica instada a dizer às mulheres a...
2796,"TSE, o relatório das Forças Armadas e o caso M..."
2797,"TSE, o relatório das Forças Armadas e o caso M..."
2798,✅ Notícias do Dia - Ano 6 n° 2183\nDia 11 de ...
